In [2]:
import os, sys
root_path = os.path.abspath('.').split('jupyters')[0]
sys.path.append(root_path)

In [3]:
import json
import argparse
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from models.cnn_geo import CNN_geo, Spatial_transformer_regressor

import simulator as sim

In [5]:
shape=(10,2)
tf.random.uniform(shape)

<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
array([[0.6045655 , 0.8963326 ],
       [0.04040706, 0.06514513],
       [0.37465954, 0.6013597 ],
       [0.36991644, 0.08753943],
       [0.5843967 , 0.5169685 ],
       [0.57698643, 0.29928684],
       [0.7742704 , 0.5436506 ],
       [0.2534114 , 0.34903502],
       [0.12029767, 0.6137092 ],
       [0.81730306, 0.11896622]], dtype=float32)>

In [17]:
def gen_motion_parameter(shape):
    while True:
        yield (tf.random.uniform(shape) - 0.5) * 2 * 0.2

In [22]:
from functools import partial
gen = partial(gen_motion_parameter, (9,2))
ds = tf.data.Dataset.from_generator(gen, output_types=tf.float32)
ds = ds.batch(16)

In [3]:
regressor = Spatial_transformer_regressor(18)

In [25]:
parameters = np.zeros([9, 2], dtype=np.float32) + np.array([[0, 0.1]])
map_size = (16, 16)
axay = np.array([[0.0, 0.0], [0.5, 0.0], [1.0, 0.0],
               [0.0, 0.5], [0.5, 0.5], [1.0, 0.5],
               [0.0, 1.0], [0.5, 1.0], [1.0, 1.0]])
axay *= (map_size[0] - 1, map_size[1] - 1)
bxby = sim.get_tgt_from_src(parameters, map_size, axay)


In [27]:
axay

array([[ 0. ,  0. ],
       [ 7.5,  0. ],
       [15. ,  0. ],
       [ 0. ,  7.5],
       [ 7.5,  7.5],
       [15. ,  7.5],
       [ 0. , 15. ],
       [ 7.5, 15. ],
       [15. , 15. ]])

In [26]:
bxby

array([[ 1,  0],
       [ 9,  0],
       [15,  0],
       [ 1,  7],
       [ 9,  7],
       [15,  7],
       [ 1, 15],
       [ 9, 15],
       [15, 15]])

In [ ]:
axybxy = np.concatenate([ij.astype(np.int32), kl], axis=-1)

correlations = sim.generate_correlations(ijkl, map_size)

In [5]:
def loss_fn(pred, label):
    loss = tf.sqrt(tf.reduce_sum(tf.pow(pred-label, 2)))
    return loss

In [6]:
map_size = (16, 16)
ij = np.array([[0.0, 0.0], [0.5, 0.0], [1.0, 0.0],
               [0.0, 0.5], [0.5, 0.5], [1.0, 0.5],
               [0.0, 1.0], [0.5, 1.0], [1.0, 1.0]])
ij *= (map_size[0] - 1, map_size[1] - 1)

optimizer = tf.keras.optimizers.SGD(
        learning_rate=0.001)

i = 0
for epoch in range(10000):
    i+= 1
    moving_vectors = (tf.random.uniform([9, 2]) - 0.5) * 2 * 0.2
    kl = sim.get_tgt_from_src(parameters, map_size, ij)
    ijkl = np.concatenate([ij.astype(np.int32), kl], axis=-1)
    correlations = sim.generate_correlations(ijkl, map_size)
    with tf.GradientTape() as tape:
        pred = regressor(correlations)
        pred = tf.reshape(pred, [9, 2])
        loss = loss_fn(pred, moving_vectors)
    gradients = tape.gradient(loss, regressor.trainable_variables)
    optimizer.apply_gradients(zip(gradients, regressor.trainable_variables)) 
    if i % 500 == 0:
        print(i, "iter loss : ", loss.numpy())
        print("parameter :", moving_vectors.numpy(), "\n", "prediction : ", pred.numpy())

500 iter loss :  0.481841
parameter : [[-0.10476589 -0.06019478]
 [ 0.06511851 -0.02079959]
 [ 0.08568378 -0.06406579]
 [-0.15959768  0.14474931]
 [ 0.1384574  -0.1628807 ]
 [-0.0371356   0.06598578]
 [-0.15719615  0.14719172]
 [ 0.08556409  0.07197385]
 [-0.05840969  0.19190823]] 
 prediction :  [[-0.01273398 -0.00415527]
 [ 0.00129735  0.01238223]
 [ 0.00923394  0.00883345]
 [ 0.00143421 -0.00645209]
 [-0.00306844  0.00758082]
 [ 0.00899942 -0.00694797]
 [ 0.00467724 -0.00403961]
 [ 0.01045948  0.00157388]
 [ 0.0166025   0.00859921]]
1000 iter loss :  0.49784645
parameter : [[ 0.08863335 -0.06727991]
 [ 0.07121658 -0.14155631]
 [ 0.03232112 -0.01790605]
 [ 0.13851872  0.07789383]
 [ 0.12474122 -0.14719577]
 [ 0.08204255  0.1606021 ]
 [-0.01088686  0.1449912 ]
 [-0.1215592  -0.19503699]
 [-0.14061204 -0.11579404]] 
 prediction :  [[-0.01178662 -0.00611689]
 [ 0.00770475  0.00351667]
 [-0.00850152  0.00152415]
 [-0.00576113 -0.01784121]
 [ 0.01691202 -0.0037808 ]
 [ 0.00775135  0.00088

8000 iter loss :  0.48718637
parameter : [[ 0.10572457  0.16085029]
 [-0.0486938  -0.04602947]
 [-0.00092268 -0.1860869 ]
 [ 0.03374667  0.1160656 ]
 [ 0.07716532  0.10669909]
 [ 0.04011183  0.07282429]
 [ 0.16545148 -0.16865993]
 [-0.05640993 -0.141253  ]
 [ 0.0906437   0.16134639]] 
 prediction :  [[-0.00577555  0.00131577]
 [-0.00280476 -0.01039101]
 [ 0.00487683  0.00957119]
 [-0.02225017 -0.00849857]
 [-0.00032489  0.00322207]
 [ 0.00439963 -0.00012288]
 [-0.00509658 -0.00234513]
 [ 0.00851502 -0.00537244]
 [ 0.00103746 -0.00705585]]
8500 iter loss :  0.5280905
parameter : [[-0.06182847  0.02931104]
 [-0.01836805 -0.19394794]
 [-0.1008019  -0.07949843]
 [-0.12960377 -0.1604745 ]
 [-0.14666724 -0.17290597]
 [ 0.14503923  0.13490443]
 [-0.13571124  0.09632297]
 [-0.14957285 -0.05711231]
 [-0.12338424  0.13615008]] 
 prediction :  [[-0.00156414 -0.00301311]
 [ 0.004589   -0.00472952]
 [ 0.01152813 -0.00919306]
 [-0.00205118  0.00475022]
 [ 0.00197312  0.00202426]
 [ 0.0035002   0.001